In [1]:
from huggingface_hub import hf_hub_download
import os

# 모델 ID 설정 (Hugging Face에서 다운로드한 모델명)
model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# 모델의 주요 파일 (config.json)을 기준으로 경로 찾기
model_cache_path = os.path.dirname(hf_hub_download(model_id, filename="config.json"))

print(f"모델의 실제 저장 경로: {model_cache_path}")

모델의 실제 저장 경로: /mnt/ssd/1/hub/models--Bllossom--llama-3.2-Korean-Bllossom-3B/snapshots/e68fbb0d9c2a4031b0d61b14014eac1a4810ac2e


In [2]:
import triton
print(triton.__file__)

/home/iljoo/.local/share/virtualenvs/sanguk-WEz904Hl/lib/python3.12/site-packages/triton/__init__.py


### 데이터셋 구축

In [3]:
import jsonlines
from datasets import Dataset

# JSONLines 파일 경로
jsonl_path = "/mnt/ssd/1/sanguk/dataset/iljoo_dataset.jsonl"

# 원하는 라인 번호
target_line = 498

# JSONLines 파일을 읽어서 데이터셋 생성
dataset_list_check = []

with jsonlines.open(jsonl_path) as f:
    for lineno, line in enumerate(f.iter(), start=1):
        if lineno == target_line:  # 원하는 라인 번호에 도달하면
            try:
                instruction = line.get("question", "")
                context = line.get("context", "")

                # response를 문자열 형태로 변환
                response_list = line.get("answers", {}).get("text", [])
                response_str = ", ".join(response_list)  # 쉼표와 공백으로 구분하여 문자열 생성

                formatted_text = f"Instruction:\n{instruction}\n\nContext:\n{context}\n\nResponse:\n{response_str}"

                dataset_list_check.append({
                    "instruction": instruction,
                    "context": context,
                    "response": response_str,  # 문자열 형태의 response 저장
                    "text": formatted_text
                })

                print(f"Line {lineno}:")  # 현재 처리 중인 line 번호 출력
                print(f"Instruction: {instruction}")
                print(f"Context: {context}")
                print(f"Response: {response_str}")  # 문자열 형태의 response 출력
                print(f"Formatted Text: {formatted_text}")
                print("-" * 20)  # 구분선 추가

            except Exception as e:
                print(f"Error at line {lineno}: {e}")

            break  # 원하는 라인을 찾았으므로 루프 종료
        elif lineno > target_line:  # target_line이 파일에 없는 경우를 대비
            print(f"Line {target_line} not found in the file.")
            break

print("데이터셋 생성 완료")

# Hugging Face Dataset으로 변환
dataset_check = Dataset.from_list(dataset_list_check)

# 데이터셋 정보 확인
print(dataset_check)

Line 498:
Instruction: 이 공식 자료를 토대로, 양사의 디지털 전환 전략은 어떻게 구분될 수 있을까요?
Context: 공식 홈페이지와 최근 블로그 포스트에 따르면, 일주지앤에스는 ERP 및 MES 시스템을 도입하여 조선해양 분야의 디지털 전환을 선도하고 있고, 티허브는 IoT 기술을 접목한 실시간 모니터링 서비스를 통해 시장 경쟁력을 강화하고 있다.
Response: ERP/MES 기반 조선해양 디지털 전환과 IoT 실시간 모니터링 서비스
Formatted Text: Instruction:
이 공식 자료를 토대로, 양사의 디지털 전환 전략은 어떻게 구분될 수 있을까요?

Context:
공식 홈페이지와 최근 블로그 포스트에 따르면, 일주지앤에스는 ERP 및 MES 시스템을 도입하여 조선해양 분야의 디지털 전환을 선도하고 있고, 티허브는 IoT 기술을 접목한 실시간 모니터링 서비스를 통해 시장 경쟁력을 강화하고 있다.

Response:
ERP/MES 기반 조선해양 디지털 전환과 IoT 실시간 모니터링 서비스
--------------------
데이터셋 생성 완료
Dataset({
    features: ['instruction', 'context', 'response', 'text'],
    num_rows: 1
})


In [4]:
import jsonlines
from datasets import Dataset

# JSONLines 파일 경로
jsonl_path = "/mnt/ssd/1/sanguk/dataset/iljoo_dataset.jsonl"

# JSONLines 파일을 읽어서 데이터셋 생성
dataset_list = []
with jsonlines.open(jsonl_path) as f:
    for lineno, line in enumerate(f.iter(), start=1):
        try:
            # 각 줄의 데이터를 이용해 템플릿에 맞는 형식의 문자열 생성
            # 여기서는 question을 instruction, answers.text[0]을 response, 그리고 context도 함께 저장합니다.
            instruction = line.get("question", "")
            context = line.get("context", "")
            
            response_list = line.get("answers", {}).get("text", [""])
            response = ", ".join(response_list)
            # 원하는 템플릿에 따라 하나의 통합된 텍스트도 생성 가능
            formatted_text = f"Instruction:\n{instruction}\n\nContext:\n{context}\n\nResponse:\n{response}"
            
            # 각 샘플을 딕셔너리 형태로 저장
            dataset_list.append({
                "instruction": instruction,
                "context": context,
                "response": response,
                "text": formatted_text
            })
        except Exception as e:
            # 문제가 있는 줄과 오류를 출력하여 확인
            print(f"Error at line {lineno}: {e}")

print("데이터셋 생성 완료")

# Hugging Face Dataset으로 변환
dataset = Dataset.from_list(dataset_list)

# 데이터셋 정보 확인
print(dataset)

데이터셋 생성 완료
Dataset({
    features: ['instruction', 'context', 'response', 'text'],
    num_rows: 500
})


In [ ]:
import os
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,
                          BitsAndBytesConfig, EarlyStoppingCallback)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import snapshot_download

import warnings
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly.")

model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"
# 전체 모델 다운로드를 위한 캐시 경로 확인
original_model_path = snapshot_download(repo_id=model_id)

# 새로운 모델 저장 경로 (bf16 환경용 경로, 추후 저장 시 사용)
new_model_path = os.path.join("/mnt/ssd/1/hub", "models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16_lr_qlr")

# 기존 모델과 토크나이저 로드 (bfloat16 사용)
tokenizer = AutoTokenizer.from_pretrained(original_model_path)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    original_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 새로운 경로가 없으면 생성 후 저장
if not os.path.exists(new_model_path):
    os.makedirs(new_model_path, exist_ok=True)
model.save_pretrained(new_model_path)
tokenizer.save_pretrained(new_model_path)
print(f"모델이 새로운 경로로 저장되었습니다: {new_model_path}")

# QLoRA: 4-bit 양자화 설정 적용하여 모델 재로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    new_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)

# k-bit 학습 준비 (QLoRA 최적화)
model = prepare_model_for_kbit_training(model)

# 특정 레이어만 학습 가능하도록 설정 (q_proj, v_proj)
for name, param in model.named_parameters():
    if "q_proj" in name or "v_proj" in name:
        param.data = param.data.to(torch.float32)  # 계산 안정성을 위해 float32 변환
        param.requires_grad = True
    else:
        param.requires_grad = False

# LoRA 설정: 일부 파라미터만 업데이트하도록 구성
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

def preprocess_data(example):
    inputs = tokenizer(
        example["instruction"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    labels = tokenizer(
        example["response"],
        truncation=True,
        padding="max_length",
        max_length=512
    )["input_ids"]
    inputs["labels"] = labels
    return inputs

fine_tuning_dataset = dataset.map(preprocess_data)

# 8:2 비율로 데이터셋 분리 (train: 80%, eval: 20%)
split_datasets = fine_tuning_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# -------------------------
# Trainer 설정 및 학습 진행
# -------------------------
training_args = TrainingArguments(
    output_dir=new_model_path,             # 모델 저장 경로
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_steps=1,
    save_total_limit=2,
    save_steps=1,
    eval_strategy="steps",
    load_best_model_at_end=True,
    fp16=False,
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=20)]
)

print("Fine-tuning 시작...")
trainer.train()
print("Fine-tuning 완료!")

trainer.save_model(new_model_path)
model.save_pretrained(new_model_path)
tokenizer.save_pretrained(new_model_path)
print(f"Fine-tuned 모델이 {new_model_path}에 저장되었습니다.")


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델이 새로운 경로로 저장되었습니다: /mnt/ssd/1/hub/models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16_lr_qlr


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Fine-tuning 시작...


Step,Training Loss,Validation Loss
1,11.899100,11.948086
2,12.342400,11.804601
3,11.826900,11.548327
4,10.810800,11.103726
5,10.599300,10.553340


In [6]:
# import os
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
# from datasets import Dataset
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# from huggingface_hub import snapshot_download

# # 모델 ID (Hugging Face에서 다운로드한 모델명)
# model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# # 캐시된 모델의 실제 경로 찾기 (전체 다운로드)
# original_model_path = snapshot_download(repo_id=model_id)

# # 새로운 모델 저장 경로 지정
# # new_model_path = os.path.expanduser("~/.cache/huggingface/hub/models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B")
# new_model_path = os.path.join("/mnt/ssd/1/hub", "models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16")

# # 기존 모델 & 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained(original_model_path)
# tokenizer.pad_token = tokenizer.eos_token 
# model = AutoModelForCausalLM.from_pretrained(original_model_path, torch_dtype=torch.bfloat16, device_map="auto")

# # 새로운 경로가 존재하는지 확인 후 저장
# if not os.path.exists(new_model_path):
#     os.makedirs(new_model_path, exist_ok=True)

# # 새로운 모델 디렉토리에 저장
# model.save_pretrained(new_model_path)
# tokenizer.save_pretrained(new_model_path)

# print(f"모델이 새로운 경로로 저장되었습니다: {new_model_path}")

# # QLoRA 적용하여 모델 로드 (4-bit 양자화)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16
# )

# model = AutoModelForCausalLM.from_pretrained(
#     new_model_path,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     quantization_config=bnb_config  # 올바른 설정 적용
# )

# # QLoRA 최적화 적용
# model = prepare_model_for_kbit_training(model)

# # 특정 레이어만 학습 가능하도록 설정 (부분 Fine-tuning 적용)
# for name, param in model.named_parameters():
#     if "q_proj" in name or "v_proj" in name:  # 일부 레이어만 학습
#         param.data = param.data.to(torch.float32)  # float 변환 
#         param.requires_grad = True  # 학습 가능하도록 설정
#     else:
#         param.requires_grad = False  # 동결 (Frozen)

# # LoRA 설정 (작은 가중치만 학습)
# lora_config = LoraConfig(
#     r=8,  # 작은 가중치 차원
#     lora_alpha=32,  # LoRA 학습률 스케일
#     target_modules=["q_proj", "v_proj"],  # 적은 파라미터만 학습
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# # 모델을 LoRA 방식으로 변환
# model = get_peft_model(model, lora_config)

# # 3개의 커스텀 데이터 (SQuAD 스타일)
# custom_data = [
#     {
#         "context": "일주지앤에스는 부산광역시 동래구 사직동 석사로 10-1 률영빌딩 5층에 본사를 두고 있는 디지털 혁신 전문기업이다. ",
#         "question": "일주지앤에스 알아?",
#         "answers": {"text": ["부산에 위치한 디지털 혁신 전문기업"], "answer_start": [8]}
#     },
#     {
#         "context": "부산 디지털 혁신 전문기업의 대표이사는 김정엽 이다. 김정엽 대표는 경북대학교 통계학과 출신으로 현대중공업 전산실에서 근무하다가 일주지앤에스를 창업하였으며, 티허브의 공동 태표이사이기도 하다.",
#         "question": "일주지앤에스 대표는?",
#         "answers": {"text": ["김정엽"], "answer_start": [22]}
#     },
#     {
#         "context": "일주지앤에스 김도현 이사는 일주지앤에스의 미래기술연구소 소장이다. 김도현 이사는 AI,빅데이터,융합기술 전문가이다.",
#         "question": "일주지앤에스 김도현 알아?",
#         "answers": {"text": ["일주지앤에스의 김도현 이사는 미래기술연구소 소장이다"], "answer_start": [0]}
#     },
#     {
#         "context": "일주지앤에스는 2006년 6월 29일에 설립된 IT 서비스 전문기업으로, 부산광역시 동래구 사직동 석사로 10-1 률영빌딩 5층에 본사를 두고 있습니다.",
#         "question": "일주지앤에스는 언제 설립되었나요?",
#         "answers": {"text": ["2006년 6월 29일"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 정보시스템 통합, 소프트웨어 구축, 정보통신공사, 컴퓨터 도매, 연구개발, 시각 디자인, 전자부품 개발 및 제조 등의 다양한 사업을 영위하고 있습니다.",
#         "question": "일주지앤에스의 주요 사업 분야는 무엇인가요?",
#         "answers": {"text": ["정보시스템 통합, 소프트웨어 구축, 정보통신공사, 컴퓨터 도매, 연구개발, 시각 디자인, 전자부품 개발 및 제조"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 스마트 공장 구축, 시스템 통합(SI), IT 아웃소싱, 솔루션 개발, 컨설팅 사업 등을 주축으로 종합 정보 서비스를 제공하고 있습니다.",
#         "question": "일주지앤에스가 제공하는 서비스는 무엇인가요?",
#         "answers": {"text": ["스마트 공장 구축, 시스템 통합(SI), IT 아웃소싱, 솔루션 개발, 컨설팅 사업"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 현대일렉트릭, 휴먼중공업, 삼강엠엔티 등과 스마트 공장 구축 프로젝트를 성공적으로 수행하였습니다.",
#         "question": "일주지앤에스가 스마트 공장 구축을 수행한 기업은 어디인가요?",
#         "answers": {"text": ["현대일렉트릭, 휴먼중공업, 삼강엠엔티"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 2024년 5월 29일 KNN 인물포커스에 소개되었으며, 김정엽 대표이사가 중대재해처벌법과 관련된 인터뷰를 진행하였습니다.",
#         "question": "일주지앤에스가 KNN 인물포커스에 소개된 날짜는 언제인가요?",
#         "answers": {"text": ["2024년 5월 29일"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 태양광 발전소 조각투자 플랫폼 '햇나'를 런칭하여 블록체인 기반의 조각투자 서비스를 제공하고 있습니다.",
#         "question": "일주지앤에스가 런칭한 태양광 발전소 조각투자 플랫폼의 이름은 무엇인가요?",
#         "answers": {"text": ["햇나"], "answer_start": [25]}
#     },
#     {
#         "context": "일주지앤에스는 미래기술연구소, DT사업부, IoT사업부, ESG사업부, SDM실, 제조ICT사업부, 대외사업부, E플랫폼사업부, 인프라사업부, 전략기획실, 경영지원실, 기술영업실로 구성되어 있습니다.",
#         "question": "일주지앤에스의 조직 구성은 어떻게 되어 있나요?",
#         "answers": {"text": ["미래기술연구소, DT사업부, IoT사업부, ESG사업부, SDM실, 제조ICT사업부, 대외사업부, E플랫폼사업부, 인프라사업부, 전략기획실, 경영지원실, 기술영업실"], "answer_start": [8]}
#     },
# ]

# # Hugging Face Dataset 객체로 변환
# dataset = Dataset.from_list(custom_data)

# # 데이터 전처리
# def preprocess_data(examples):
#     inputs = tokenizer(examples["question"], examples["context"], truncation=True, padding="max_length", max_length=512)
#     inputs["labels"] = tokenizer(examples["answers"]["text"][0], truncation=True, padding="max_length", max_length=512)["input_ids"]
#     return inputs

# fine_tuning_dataset = dataset.map(preprocess_data)

# # 학습 하이퍼파라미터 설정
# training_args = TrainingArguments(
#     output_dir=new_model_path,  # 새로운 모델 덮어쓰기
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     num_train_epochs=10,
#     logging_steps=1,
#     save_total_limit=2,
#     save_steps=1,
#     eval_strategy="steps",
#     load_best_model_at_end=True,
#     fp16=True,  # GPU 최적화
# )

# # Trainer 설정
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=fine_tuning_dataset,
#     eval_dataset=fine_tuning_dataset,
#     processing_class=tokenizer,
# )

# print("Fine-tuning 시작...")
# trainer.train()
# print("Fine-tuning 완료!")

# # 학습된 LoRA 모델 저장
# trainer.save_model(new_model_path)  # 전체 모델 저장
# model.save_pretrained(new_model_path)  # LoRA 가중치 저장
# tokenizer.save_pretrained(new_model_path)

# print(f"Fine-tuned 모델이 {new_model_path}에 저장되었습니다.")

In [7]:
# import os
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, EarlyStoppingCallback
# from datasets import Dataset
# from huggingface_hub import snapshot_download

# # 모델 ID (Hugging Face에서 다운로드한 모델명)
# model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# # 캐시된 모델의 실제 경로 찾기 (전체 다운로드)
# original_model_path = snapshot_download(repo_id=model_id)

# # 새로운 모델 저장 경로 지정
# # new_model_path = os.path.expanduser("~/.cache/huggingface/hub/models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B")
# new_model_path = os.path.join("/mnt/ssd/1/hub", "models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16")

# # 기존 모델 & 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained(original_model_path)
# tokenizer.pad_token = tokenizer.eos_token 
# model = AutoModelForCausalLM.from_pretrained(
#     original_model_path,
#     torch_dtype=torch.float32,
#     device_map="auto"
# )

# # 새로운 경로가 존재하는지 확인 후 저장
# if not os.path.exists(new_model_path):
#     os.makedirs(new_model_path, exist_ok=True)

# # 모델과 토크나이저를 새로운 경로에 저장
# model.save_pretrained(new_model_path)
# tokenizer.save_pretrained(new_model_path)

# print(f"모델이 새로운 경로로 저장되었습니다: {new_model_path}")

# # 풀 튜닝을 위해 전체 파라미터 업데이트 (모델 재로드)
# model = AutoModelForCausalLM.from_pretrained(
#     new_model_path,
#     torch_dtype=torch.float32,
#     device_map="auto"
# )

# # 커스텀 데이터 (SQuAD 스타일 예시)
# custom_data = [
#     {
#         "context": "일주지앤에스는 부산광역시 동래구 사직동 석사로 10-1 률영빌딩 5층에 본사를 두고 있는 디지털 혁신 전문기업이다. ",
#         "question": "일주지앤에스 알아?",
#         "answers": {"text": ["부산에 위치한 디지털 혁신 전문기업"], "answer_start": [8]}
#     },
#     {
#         "context": "부산 디지털 혁신 전문기업의 대표이사는 김정엽 이다. 김정엽 대표는 경북대학교 통계학과 출신으로 현대중공업 전산실에서 근무하다가 일주지앤에스를 창업하였으며, 티허브의 공동 태표이사이기도 하다.",
#         "question": "일주지앤에스 대표는?",
#         "answers": {"text": ["김정엽"], "answer_start": [22]}
#     },
#     {
#         "context": "일주지앤에스 김도현 이사는 일주지앤에스의 미래기술연구소 소장이다. 김도현 이사는 AI,빅데이터,융합기술 전문가이다.",
#         "question": "일주지앤에스 김도현 알아?",
#         "answers": {"text": ["일주지앤에스의 김도현 이사는 미래기술연구소 소장이다"], "answer_start": [0]}
#     },
#     {
#         "context": "일주지앤에스는 2006년 6월 29일에 설립된 IT 서비스 전문기업으로, 부산광역시 동래구 사직동 석사로 10-1 률영빌딩 5층에 본사를 두고 있습니다.",
#         "question": "일주지앤에스는 언제 설립되었나요?",
#         "answers": {"text": ["2006년 6월 29일"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 정보시스템 통합, 소프트웨어 구축, 정보통신공사, 컴퓨터 도매, 연구개발, 시각 디자인, 전자부품 개발 및 제조 등의 다양한 사업을 영위하고 있습니다.",
#         "question": "일주지앤에스의 주요 사업 분야는 무엇인가요?",
#         "answers": {"text": ["정보시스템 통합, 소프트웨어 구축, 정보통신공사, 컴퓨터 도매, 연구개발, 시각 디자인, 전자부품 개발 및 제조"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 스마트 공장 구축, 시스템 통합(SI), IT 아웃소싱, 솔루션 개발, 컨설팅 사업 등을 주축으로 종합 정보 서비스를 제공하고 있습니다.",
#         "question": "일주지앤에스가 제공하는 서비스는 무엇인가요?",
#         "answers": {"text": ["스마트 공장 구축, 시스템 통합(SI), IT 아웃소싱, 솔루션 개발, 컨설팅 사업"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 현대일렉트릭, 휴먼중공업, 삼강엠엔티 등과 스마트 공장 구축 프로젝트를 성공적으로 수행하였습니다.",
#         "question": "일주지앤에스가 스마트 공장 구축을 수행한 기업은 어디인가요?",
#         "answers": {"text": ["현대일렉트릭, 휴먼중공업, 삼강엠엔티"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 2024년 5월 29일 KNN 인물포커스에 소개되었으며, 김정엽 대표이사가 중대재해처벌법과 관련된 인터뷰를 진행하였습니다.",
#         "question": "일주지앤에스가 KNN 인물포커스에 소개된 날짜는 언제인가요?",
#         "answers": {"text": ["2024년 5월 29일"], "answer_start": [8]}
#     },
#     {
#         "context": "일주지앤에스는 태양광 발전소 조각투자 플랫폼 '햇나'를 런칭하여 블록체인 기반의 조각투자 서비스를 제공하고 있습니다.",
#         "question": "일주지앤에스가 런칭한 태양광 발전소 조각투자 플랫폼의 이름은 무엇인가요?",
#         "answers": {"text": ["햇나"], "answer_start": [25]}
#     },
#     {
#         "context": "일주지앤에스는 미래기술연구소, DT사업부, IoT사업부, ESG사업부, SDM실, 제조ICT사업부, 대외사업부, E플랫폼사업부, 인프라사업부, 전략기획실, 경영지원실, 기술영업실로 구성되어 있습니다.",
#         "question": "일주지앤에스의 조직 구성은 어떻게 되어 있나요?",
#         "answers": {"text": ["미래기술연구소, DT사업부, IoT사업부, ESG사업부, SDM실, 제조ICT사업부, 대외사업부, E플랫폼사업부, 인프라사업부, 전략기획실, 경영지원실, 기술영업실"], "answer_start": [8]}
#     },
# ]

# # Hugging Face Dataset 객체로 변환
# dataset = Dataset.from_list(custom_data)

# # 데이터 전처리: 질문과 컨텍스트를 입력으로, 답변 텍스트를 라벨로 설정
# def preprocess_data(examples):
#     inputs = tokenizer(
#         examples["question"],
#         examples["context"],
#         truncation=True,
#         padding="max_length",
#         max_length=512
#     )
#     # 답변 텍스트를 라벨로 변환 (토큰 ID 형태)
#     inputs["labels"] = tokenizer(
#         examples["answers"]["text"][0],
#         truncation=True,
#         padding="max_length",
#         max_length=512
#     )["input_ids"]
#     return inputs

# full_tuning_dataset = dataset.map(preprocess_data)

# # 학습 하이퍼파라미터 설정
# training_args = TrainingArguments(
#     output_dir=new_model_path,             # 저장 경로
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     num_train_epochs=10,
#     logging_steps=1,
#     save_total_limit=2,
#     save_steps=1,
#     eval_strategy="steps",
#     load_best_model_at_end=True,
#     fp16=False,                            
#     bf16=True
# )

# # Trainer 설정 (전체 모델의 파라미터 업데이트)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=full_tuning_dataset,
#     eval_dataset=full_tuning_dataset,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=100)] 
# )

# print("Full tuning 시작...")
# trainer.train()
# print("Full tuning 완료!")

# # 학습된 전체 모델 저장
# trainer.save_model(new_model_path)
# model.save_pretrained(new_model_path)
# tokenizer.save_pretrained(new_model_path)

# print(f"Full tuned 모델이 {new_model_path}에 저장되었습니다.")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델이 새로운 경로로 저장되었습니다: /mnt/ssd/1/hub/models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Full tuning 시작...


Step,Training Loss,Validation Loss
1,8.605300,0.753528
2,0.672300,0.805411
3,0.155500,0.817028
4,0.489800,0.784692
5,1.486500,0.509490
6,0.696200,1.181076
7,0.996300,0.741505
8,1.373200,0.434533
9,0.215000,0.570164
10,0.190200,0.551180


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Full tuning 완료!
Full tuned 모델이 /mnt/ssd/1/hub/models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16에 저장되었습니다.
